In [1]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("teste").getOrCreate()

In [2]:
df_vra=spark.read.parquet('arquivos/res/vra/vra_final.snappy.parquet')
df_vra.createOrReplaceTempView('vra')

In [3]:
df_aerodromos=spark.read.parquet('arquivos/har/aerodromos/aerodromos.snappy.parquet')
df_aerodromos.createOrReplaceTempView('aerodromos')

In [4]:
df_empresas=spark.read.parquet('arquivos/har/empresas/empresas.snappy.parquet')
df_empresas.createOrReplaceTempView('empresas')

In [5]:
df_vra.show()

+------------------+----------+------------------+-----------------+---------------------+----------------------+---------------------+-----------------+---------------------+-----------------+------------+--------------------+-------+-------+--------------+
|icao_empresa_aerea|numero_voo|codigo_autorizacao|codigo_tipo_linha|icao_aerodromo_origem|icao_aerodromo_destino|data_partida_prevista|data_partida_real|data_chegada_prevista|data_chegada_real|situacao_voo|codigo_justificativa|ano_voo|mes_voo|arquivo_origem|
+------------------+----------+------------------+-----------------+---------------------+----------------------+---------------------+-----------------+---------------------+-----------------+------------+--------------------+-------+-------+--------------+
|               TAM|      4660|                 0|                R|                 SBCX|                  SBCT|     01/01/2000 05:45|             null|     01/01/2000 06:37|             null|   CANCELADO|                 

In [6]:
df_empresas.show()

+---+--------------------+-------+----+----+------------------+--------------+-----+-----------------+
| id|                nome|apelido|iata|icao|indicativo_chamada|          pais|ativo|__index_level_0__|
+---+--------------------+-------+----+----+------------------+--------------+-----+-----------------+
|  2|         135 Airways|   null|null| GNL|           GENERAL| United States|    N|                0|
|  3|       1Time Airline|   null|  1T| RNX|           NEXTIME|  South Africa|    S|                1|
|  4|2 Sqn No 1 Elemen...|   null|null| WYT|              null|United Kingdom|    N|                2|
|  5|     213 Flight Unit|   null|null| TFU|              null|        Russia|    N|                3|
|  6|223 Flight Unit S...|   null|null| CHD|    CHKALOVSK-AVIA|        Russia|    N|                4|
|  7|   224th Flight Unit|   null|null| TTF|        CARGO UNIT|        Russia|    N|                5|
|  8|         247 Jet Ltd|   null|null| TWF|      CLOUD RUNNER|United Kin

In [7]:
query = (
    ' SELECT ano_voo, mes_voo, icao_empresa_aerea, icao_aerodromo_origem as aerodromo, COUNT(1) as quantidade_voos_total'
    ' FROM vra'
    ' WHERE icao_empresa_aerea IS NOT NULL'
    ' GROUP BY ano_voo, mes_voo, icao_empresa_aerea, icao_aerodromo_origem'
    ' ORDER BY quantidade_voos_total DESC'
)
resultado = spark.sql(query)
resultado.createOrReplaceTempView('voos_por_empresas')
resultado.show()

+-------+-------+------------------+---------+---------------------+
|ano_voo|mes_voo|icao_empresa_aerea|aerodromo|quantidade_voos_total|
+-------+-------+------------------+---------+---------------------+
|   2020|     02|               TAM|     SBGR|                18956|
|   2020|     02|               AZU|     SBKP|                15856|
|   2020|     03|               TAM|     SBGR|                15292|
|   2020|     02|               GLO|     SBGR|                13620|
|   2020|     02|               TAM|     SBSP|                12472|
|   2020|     02|               GLO|     SBSP|                12380|
|   2020|     03|               AZU|     SBKP|                12096|
|   2020|     03|               GLO|     SBGR|                11440|
|   2020|     03|               GLO|     SBSP|                10144|
|   2020|     02|               AZU|     SBCF|                 9940|
|   2020|     03|               TAM|     SBSP|                 9408|
|   2020|     02|               TA

In [8]:
query = (
    ' SELECT ano_voo, mes_voo, icao_empresa_aerea, icao_aerodromo_origem as aerodromo, COUNT(1) as quantidade_voos_cancelados'
    ' FROM vra'
    ' WHERE situacao_voo = "CANCELADO"'
    ' AND icao_empresa_aerea IS NOT NULL'
    ' GROUP BY ano_voo, mes_voo, icao_empresa_aerea, icao_aerodromo_origem'
    ' ORDER BY quantidade_voos_cancelados DESC'
)
resultado = spark.sql(query)
resultado.createOrReplaceTempView('voos_por_empresas_cancelado')
resultado.show()

+-------+-------+------------------+---------+--------------------------+
|ano_voo|mes_voo|icao_empresa_aerea|aerodromo|quantidade_voos_cancelados|
+-------+-------+------------------+---------+--------------------------+
|   2003|     03|               TAM|     SBSP|                      2281|
|   2003|     04|               TAM|     SBSP|                      2241|
|   2003|     01|               TAM|     SBSP|                      2080|
|   2002|     12|               TAM|     SBSP|                      1916|
|   2002|     10|               TAM|     SBSP|                      1801|
|   2003|     02|               TAM|     SBSP|                      1702|
|   2006|     08|               VRG|     SBGR|                      1679|
|   2002|     11|               TAM|     SBSP|                      1624|
|   2006|     07|               VRG|     SBGR|                      1538|
|   2007|     01|               VRN|     SBGR|                      1431|
|   2007|     08|               TAM|  

In [9]:
query = (
    ' SELECT voos_por_empresas.ano_voo, voos_por_empresas.mes_voo, voos_por_empresas.icao_empresa_aerea,'
    ' voos_por_empresas.aerodromo as aerodromo, quantidade_voos_total, quantidade_voos_cancelados'
    ' FROM voos_por_empresas'
    ' INNER JOIN voos_por_empresas_cancelado'
    ' ON voos_por_empresas.ano_voo = voos_por_empresas_cancelado.ano_voo'
    ' AND voos_por_empresas.mes_voo = voos_por_empresas_cancelado.mes_voo'
    ' AND voos_por_empresas.icao_empresa_aerea = voos_por_empresas_cancelado.icao_empresa_aerea'
    ' AND voos_por_empresas.aerodromo = voos_por_empresas_cancelado.aerodromo'
)
resultado = spark.sql(query)
resultado.createOrReplaceTempView('voos_por_empresas_totais')
resultado.show()

+-------+-------+------------------+---------+---------------------+--------------------------+
|ano_voo|mes_voo|icao_empresa_aerea|aerodromo|quantidade_voos_total|quantidade_voos_cancelados|
+-------+-------+------------------+---------+---------------------+--------------------------+
|   2000|     01|               AZA|     SBGL|                   12|                         3|
|   2000|     02|               PEP|     SBBE|                  260|                        55|
|   2000|     02|               PEP|     SNMA|                   50|                        50|
|   2000|     04|               AZA|     LIMC|                   32|                         2|
|   2000|     04|               VRG|     LPPT|                   32|                         2|
|   2000|     04|               BLC|     SBVT|                   25|                         1|
|   2000|     04|               NES|     SGES|                    1|                         1|
|   2000|     05|               RSL|    

In [18]:
query = (
 ' SELECT ano_voo, mes_voo, icao_empresa_aerea, nome, pais, aerodromo, quantidade_voos_total, quantidade_voos_cancelados'
    ' FROM voos_por_empresas_totais'
    ' INNER JOIN empresas'
    ' ON icao_empresa_aerea = icao'
    ' AND icao IS NOT NULL'
)
resultado = spark.sql(query)
resultado.createOrReplaceTempView('voos_por_empresas_final')
resultado.show()

+-------+-------+------------------+--------------------+-------+---------+---------------------+--------------------------+
|ano_voo|mes_voo|icao_empresa_aerea|                nome|   pais|aerodromo|quantidade_voos_total|quantidade_voos_cancelados|
+-------+-------+------------------+--------------------+-------+---------+---------------------+--------------------------+
|   2000|     01|               AZA|            Alitalia|  Italy|     SBGL|                   12|                         3|
|   2000|     04|               AZA|            Alitalia|  Italy|     LIMC|                   32|                         2|
|   2000|     04|               VRG| Varig Linhas Aereas| Brazil|     LPPT|                   32|                         2|
|   2000|     04|               BLC|          Bellesavia|Belarus|     SBVT|                   25|                         1|
|   2000|     04|               NES|Nordeste Linhas A...| Brazil|     SGES|                    1|                         1|


In [11]:
query = (
    ' SELECT ano_voo, ((SUM(quantidade_voos_cancelados) * 100) / SUM(quantidade_voos_total)) as porcentage_cancelado'
    ' FROM voos_por_empresas_totais'
    ' GROUP BY ano_voo'
    ' ORDER BY porcentage_cancelado DESC'
)
resultado = spark.sql(query)
resultado.show()

+-------+--------------------+
|ano_voo|porcentage_cancelado|
+-------+--------------------+
|   2003|  21.639278987215945|
|   2007|   20.79376927435563|
|   2006|  17.364403453142877|
|   2002|   16.09734708898827|
|   2004|  15.147583127352032|
|   2015|  14.731952647891795|
|   2005|  14.328470374399776|
|   2016|  13.038197913655527|
|   2014|  12.829725685378257|
|   2001|  11.937893464068768|
|   2008|  11.901247245576878|
|   2017|   11.39345551198876|
|   2009|  11.214490003267345|
|   2010|  11.100775102805786|
|   2013|  10.972889880908216|
|   2000|  10.870579110258314|
|   2011|   10.64843629802649|
|   2012|   9.907749499910539|
|   2022|   6.645185073933999|
|   2020|  5.3335354133799955|
+-------+--------------------+
only showing top 20 rows



In [12]:
resultado.describe().show()

+-------+-----------------+--------------------+
|summary|          ano_voo|porcentage_cancelado|
+-------+-----------------+--------------------+
|  count|               23|                  23|
|   mean|           2011.0|  11.737544732449168|
| stddev|6.782329983125314|   4.854355024092584|
|    min|             2000|  2.1767853564112505|
|    max|             2022|  21.639278987215945|
+-------+-----------------+--------------------+



In [13]:
query = (
    ' SELECT ano_voo, SUM(quantidade_voos_total) as quantidade_voos_total, SUM(quantidade_voos_cancelados) as quantidade_voos_cancelados' 
    ' FROM voos_por_empresas_totais'
    ' WHERE ano_voo = 2003'
    ' GROUP BY ano_voo'
)
resultado = spark.sql(query)
resultado.show()

+-------+---------------------+--------------------------+
|ano_voo|quantidade_voos_total|quantidade_voos_cancelados|
+-------+---------------------+--------------------------+
|   2003|               708226|                    153255|
+-------+---------------------+--------------------------+



In [14]:
resultado.describe().show()

+-------+-------+---------------------+--------------------------+
|summary|ano_voo|quantidade_voos_total|quantidade_voos_cancelados|
+-------+-------+---------------------+--------------------------+
|  count|      1|                    1|                         1|
|   mean| 2003.0|             708226.0|                  153255.0|
| stddev|   null|                 null|                      null|
|    min|   2003|               708226|                    153255|
|    max|   2003|               708226|                    153255|
+-------+-------+---------------------+--------------------------+



In [15]:
query = (
    ' SELECT *'
    ' FROM voos_por_empresas_totais'
    ' WHERE icao_empresa_aerea IS NULL'
)
resultado = spark.sql(query)
resultado.show()

+-------+-------+------------------+---------+---------------------+--------------------------+
|ano_voo|mes_voo|icao_empresa_aerea|aerodromo|quantidade_voos_total|quantidade_voos_cancelados|
+-------+-------+------------------+---------+---------------------+--------------------------+
+-------+-------+------------------+---------+---------------------+--------------------------+



In [16]:
query = (
    ' SELECT *'
    ' FROM empresas'
    ' WHERE icao IS NULL'
)
resultado = spark.sql(query)
resultado.show()

+----+--------------------+-------+----+----+-------------------+--------------------+-----+-----------------+
|  id|                nome|apelido|iata|icao| indicativo_chamada|                pais|ativo|__index_level_0__|
+----+--------------------+-------+----+----+-------------------+--------------------+-----+-----------------+
|  14|Abacus International|   null|  1B|null|               null|           Singapore|    S|               12|
|  66|Advance Leasing C...|   null|  4G|null|               null|       United States|    N|               64|
|  67|Aztec Worldwide A...|   null|  7A|null|               null|       United States|    N|               65|
|  68|           Air Tindi|   null|  8T|null|               null|Canadian Territories|    S|               66|
| 146|          Air Salone|   null|  2O|null|               null|        Sierra Leone|    S|              144|
| 299|          Air Libert|   null|  VD|null|               null|              France|    S|              297|
|

In [45]:
query = (
    ' WITH ano_empresa as ('
    ' SELECT ano_voo, icao_empresa_aerea, SUM(quantidade_voos_total) as quantidade_voos_total, SUM(quantidade_voos_cancelados) as quantidade_voos_cancelados'
    ' FROM voos_por_empresas_final'
    ' WHERE icao_empresa_aerea IN ("TAM", "GLO", "AZU")'
    ' GROUP BY ano_voo, icao_empresa_aerea'
    ')'
    'SELECT ano_voo, icao_empresa_aerea, nome, quantidade_voos_total, quantidade_voos_cancelados, '
    '  ((quantidade_voos_cancelados * 100) / quantidade_voos_total) as porcentage_cancelado'
    ' FROM ano_empresa'
    ' INNER JOIN empresas'
    ' ON icao = icao_empresa_aerea'
    ' ORDER BY ano_voo ASC'
)
resultado = spark.sql(query)
resultado.show()

+-------+------------------+--------------------+---------------------+--------------------------+--------------------+
|ano_voo|icao_empresa_aerea|                nome|quantidade_voos_total|quantidade_voos_cancelados|porcentage_cancelado|
+-------+------------------+--------------------+---------------------+--------------------------+--------------------+
|   2000|               TAM|TAM Brazilian Air...|               120430|                      9044|   7.509756705139916|
|   2001|               GLO|Gol Transportes A...|                23898|                      2141|   8.958908695288308|
|   2001|               TAM|TAM Brazilian Air...|               250234|                     22378|   8.942829511577164|
|   2002|               GLO|Gol Transportes A...|                53331|                      4306|   8.074103242016838|
|   2002|               TAM|TAM Brazilian Air...|               281836|                     51760|   18.36529045260364|
|   2003|               TAM|TAM Brazilia

In [50]:
cancelamento_ano_empresa = resultado.toPandas()

In [51]:
cancelamento_ano_empresa_tam = cancelamento_ano_empresa[cancelamento_ano_empresa['icao_empresa_aerea'] == 'TAM']

,ano_voo,icao_empresa_aerea,nome,quantidade_voos_total,quantidade_voos_cancelados,porcentage_cancelado
0,2000,TAM,TAM Brazilian Airlines,120430,9044,7.509757
1,2001,TAM,TAM Brazilian Airlines,250234,22378,8.942830
4,2002,TAM,TAM Brazilian Airlines,281836,51760,18.365290
6,2003,TAM,TAM Brazilian Airlines,211446,67713,32.023779
8,2004,TAM,TAM Brazilian Airlines,157952,13662,8.649463
10,2005,TAM,TAM Brazilian Airlines,212344,17575,8.276664
11,2006,TAM,TAM Brazilian Airlines,247103,19849,8.032683
13,2007,TAM,TAM Brazilian Airlines,290342,41526,14.302443
16,2008,TAM,TAM Brazilian Airlines,269480,12172,4.516847
18,2009,TAM,TAM Brazilian Airlines,293166,13156,4.487560


In [54]:
cancelamento_ano_empresa_gol = cancelamento_ano_empresa[cancelamento_ano_empresa['icao_empresa_aerea'] == 'GLO']
cancelamento_ano_empresa_gol

,ano_voo,icao_empresa_aerea,nome,quantidade_voos_total,quantidade_voos_cancelados,porcentage_cancelado
2,2001,GLO,Gol Transportes Aéreos,23898,2141,8.958909
3,2002,GLO,Gol Transportes Aéreos,53331,4306,8.074103
5,2003,GLO,Gol Transportes Aéreos,81238,9120,11.226273
7,2004,GLO,Gol Transportes Aéreos,92577,7568,8.174817
9,2005,GLO,Gol Transportes Aéreos,127443,12072,9.472470
12,2006,GLO,Gol Transportes Aéreos,172447,14115,8.185124
14,2007,GLO,Gol Transportes Aéreos,235838,33344,14.138519
15,2008,GLO,Gol Transportes Aéreos,222120,15217,6.850801
17,2009,GLO,Gol Transportes Aéreos,277338,23041,8.307913
20,2010,GLO,Gol Transportes Aéreos,315639,23766,7.529488


In [58]:
cancelamento_ano_empresa_azul = cancelamento_ano_empresa[cancelamento_ano_empresa['icao_empresa_aerea'] == 'AZU']
cancelamento_ano_empresa_azul

,ano_voo,icao_empresa_aerea,nome,quantidade_voos_total,quantidade_voos_cancelados,porcentage_cancelado
19,2009,AZU,Azul,19429,234,1.204385
22,2010,AZU,Azul,46329,1516,3.272248
23,2011,AZU,Azul,94024,5232,5.564537
27,2012,AZU,Azul,140747,11241,7.986671
30,2013,AZU,Azul,187997,15199,8.084703
32,2014,AZU,Azul,266790,34163,12.805203
37,2015,AZU,Azul,322863,47506,14.713981
38,2016,AZU,Azul,290249,34692,11.952496
42,2017,AZU,Azul,306340,50189,16.383430
46,2018,AZU,Azul,269507,19944,7.400179


In [36]:
query = (
    ' SELECT icao_empresa_aerea, SUM(quantidade_voos_total) as quantidade_voos_total'
    ' FROM voos_por_empresas_final'
    ' GROUP BY icao_empresa_aerea'
    ' ORDER BY quantidade_voos_total DESC '
    ' LIMIT 20'
)
resultado = spark.sql(query)
resultado.show()

+------------------+---------------------+
|icao_empresa_aerea|quantidade_voos_total|
+------------------+---------------------+
|               TAM|              5268355|
|               GLO|              4582792|
|               AZU|              2681396|
|               VRG|              1041727|
|               ONE|               774534|
|               TIB|               696783|
|               VSP|               423191|
|               RSL|               374665|
|               PTB|               291144|
|               WEB|               199934|
|               PTN|               191044|
|               VRN|               172930|
|               NES|               166955|
|               TTL|               163303|
|               AAL|               141711|
|               ARG|               130122|
|               TUS|               102037|
|               TBA|                89994|
|               TAP|                88728|
|               BLC|                86032|
+----------